# Hypotéza 1
*Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako na dálnicích.*

Pro řešení používáme chí-kvadrát test, nulová a alternativní hypotéza tedy budou:
- H0: Počet úmrtí při nehodách nezávisí na třídě silnice.
- H1: Počet úmrtí při nehodách závisí na třídě silnice.

Pro výpočet využijeme knihovny `pandas`, `numpy` a `scipy.stats`.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats

Načteme si předpřipravený `DataFrame`

In [ ]:
df = pd.read_pickle("accidents.pkl.gz")

Z dataframu vyfiltrujeme nehody mimo dálnice a silnice první třídy (položka `p36`), vybereme do nového dataframu
a převedeme položku `p13a` na informaci, zda nehoda byla smrtelná nebo ne.

In [ ]:
df = df[df["p36"].isin([0, 1])]
simplified = pd.DataFrame({
    "highway": df["p36"] == 0,
    "fatal": df["p13a"] > 0
})

Vytvoříme si kontingenční tabulku

In [ ]:
table = pd.crosstab(simplified["highway"], simplified["fatal"])
table

Vypočteme p-hodnotu xi2 testu pro tuto tabulku pomocí `scipy`

In [ ]:
stats = scipy.stats.chi2_contingency(table)
pval = stats[1]
print("p-hodnota:", pval)

Vypočtená hodnota `3.6e-10` je výrazně menší, než hladina významnosti `0.05`, proto nulovou hypotézu zamítáme a můžeme prohlásit,
že počet úmrtí *závisí* na třídě silnice a tedy že na silnicích první třídy se *neumíralo* se stejnou pravděpodobností jako na dálnicicích.

Dále můžeme srovnat hodnoty z kontingeční tabulky výše s `expected` částí výstupu funkce `chi2_contingency`.

In [ ]:
stats[3]

Na první pohled vidíme, že reálné hodnoty u silnic 1. třídy jsou vyšší, než očekávané. Můžeme tedy říct, že nehody
na silnicích 1. třídy *vedly* častěji k fatální nehodě.